In [ ]:
# !pip install hana_ml --upgrade

In [ ]:
import hana_ml

In [ ]:
print(hana_ml.__version__)

In [ ]:
from hana_ml import dataframe
from hana_ml.dataframe import ConnectionContext

In [ ]:
conn = dataframe.ConnectionContext(address='5ba35c1e-58ff-44a3-859a-f6715c4d6861.hana.prod-jp10.hanacloud.ondemand.com', 
                                   port=443,
                                   user='HANAML', 
                                   password='Welcome01')

In [ ]:
conn.connection.isconnected()

In [ ]:
import pandas as pd

In [ ]:
df_train=pd.read_csv('iris_train_data_tbl.csv')

In [ ]:
df_train

In [ ]:
dataframe.create_dataframe_from_pandas(conn, df_train, 'IRIS_TRAIN_TBL_00', 'HANAML', force=True, replace=True)

In [ ]:
hdf_train = conn.table('IRIS_TRAIN_TBL_00')

In [ ]:
hdf_train.head(5).collect()

In [ ]:
hdf_train.count()

In [ ]:
# https://blogs.sap.com/2019/08/21/conquer-your-data-with-the-hana-dataframe-exploratory-data-analysis/
from hana_ml.visualizers.eda import EDAVisualizer
import matplotlib.pyplot as plt
import time
import numpy as np
from hana_ml.algorithms.pal import stats

In [ ]:
f = plt.figure(figsize=(10,10))
ax1 = f.add_subplot(111)
start = time.time()
eda = EDAVisualizer(ax1)
ax, pie_data = eda.pie_plot(data = hdf_train, column = "SPECIES", title = "SPECIES", legend = True)
plt.show()
end=time.time()
print("Time: {}s".format(round(end-start),2))

In [ ]:
f = plt.figure(figsize=(15,8))
ax1 = f.add_subplot(111)
eda = EDAVisualizer(ax1)
ax, dist_data = eda.distribution_plot(data = hdf_train, column = "SEPALLENGTHCM", bins = 1, debrief=True)
plt.show()

In [ ]:
f = plt.figure(figsize=(12,8))
ax1 = f.add_subplot(111)
eda = EDAVisualizer(ax1)
ax, corr_data = eda.correlation_plot(data = hdf_train, corr_cols = ["SEPALLENGTHCM", "SEPALWIDTHCM", "PETALLENGTHCM", "PETALWIDTHCM"], 
                                     label=True, cmap="Reds")
plt.show()

In [ ]:
f = plt.figure(figsize=(15,8))
ax1 = f.add_subplot(111)
eda = EDAVisualizer(ax1)
ax, bar_data = eda.bar_plot(data = hdf_train, column='SPECIES', aggregation={'SEPALLENGTHCM': 'avg'})
plt.show()

In [ ]:
from hana_ml.algorithms.pal import trees

In [ ]:
rfc = trees.RandomForestClassifier(n_estimators=3,
                                   max_features=3,
                                   random_state=2,
                                   split_threshold=0.00001,
                                   calculate_oob=True,
                                   min_samples_leaf=1, thread_ratio=0.5)

In [ ]:
features = ['SEPALLENGTHCM','SEPALWIDTHCM','PETALLENGTHCM','PETALWIDTHCM']
label = 'SPECIES'

In [ ]:
rfc.fit(hdf_train, 'ID', features, label)

In [ ]:
df_test=pd.read_csv('iris_test_data_tbl.csv')

In [ ]:
df_test

In [ ]:
dataframe.create_dataframe_from_pandas(conn, df_test, 'IRIS_TEST_TBL_00', 'HANAML', force=True, replace=True)

In [ ]:
hdf_test = conn.table('IRIS_TEST_TBL_00')

In [ ]:
hdf_test.count()

In [ ]:
predict_df = rfc.predict(hdf_test, 'ID', features, verbose=False)

In [ ]:
predict_df.collect()

In [ ]:
rfc.score(hdf_test, 'ID', features, label)